In [9]:
import numpy as np
from trained_untrained_results_funcs import loop_through_datasets, load_mean_sem_perf, custom_add_2d
from matplotlib import pyplot as plt

In [10]:
import numpy as np
from scipy.stats import false_discovery_control

def compute_voxel_pvalues(voxel_performance, null_distribution):
    """
    Compute one-sided p-values for voxel performance against a null distribution and apply FDR correction.
    
    Parameters:
    - voxel_performance: 1D array of shape (num_voxels,) containing the voxel performance values.
    - null_distribution: 2D array of shape (1000, num_voxels) representing the null distribution for each voxel.
    
    Returns:
    - p_values: 1D array of uncorrected p-values for each voxel.
    - fdr_corrected_p_values: 1D array of FDR-corrected p-values for each voxel.
    """
    num_voxels = voxel_performance.shape[0]
    p_values = np.zeros(num_voxels)

    # Compute p-values for each voxel
    for i in range(num_voxels):
        null_dist = null_distribution[:, i]
        p_values[i] = np.nanmean(null_dist >= voxel_performance[i])  # One-sided test
        
    fdr_corrected_p_values  = false_discovery_control(p_values)
    
        # Compute and print fractions under 0.05
    frac_uncorrected = np.mean(p_values < 0.05) * 100
    frac_fdr_corrected = np.mean(fdr_corrected_p_values < 0.05) * 100 

    print(f"Fraction of uncorrected p-values < 0.05: {frac_uncorrected:.3f}")
    print(f"Fraction of FDR-corrected p-values < 0.05: {frac_fdr_corrected:.3f}")


    return p_values, fdr_corrected_p_values

In [11]:
exp = ['384', '243']
data_processed_folder_pereira = '/data/LLMs/data_processed/pereira/dataset/'

br_labels_dict = {}
num_vox_dict = {}
subjects_dict = {}
for e in exp:

    bre = np.load(f'{data_processed_folder_pereira}/networks_{e}.npy', allow_pickle=True)
    br_labels_dict[e] = bre
    num_vox_dict[e] = bre.shape[0]
    subjects_dict[e] = np.load(f"{data_processed_folder_pereira}/subjects_{e}.npy", allow_pickle=True)
    
lang_indices_dict = {}
lang_indices_384 = np.argwhere(br_labels_dict['384'] == 'language').squeeze()
lang_indices_243 = np.argwhere(br_labels_dict['243'] == 'language').squeeze()
lang_indices_dict['384'] = lang_indices_384
lang_indices_dict['243'] = lang_indices_243

subjects_arr_pereira = np.load(f"{data_processed_folder_pereira}/subjects_complete.npy", allow_pickle=True)
networks_arr_pereira = np.load(f"{data_processed_folder_pereira}/network_complete.npy", allow_pickle=True)
non_nan_indices_243 = np.load(f"{data_processed_folder_pereira}/non_nan_indices_243.npy") # voxels which are in 243
non_nan_indices_384 = np.load(f"{data_processed_folder_pereira}/non_nan_indices_384.npy") # voxels which are in 384
non_nan_indices_dict = {'384': non_nan_indices_384, '243': non_nan_indices_243}
lang_indices = np.argwhere(networks_arr_pereira=='language').squeeze()

In [12]:
store_perf = []

perf = 'out_of_sample_r2'

for dataset in ['pereira', 'fedorenko', 'blank']:
    
    for fe in ['', '-mp', '-sp']:
    
        all_gauss = []
        
        simple_perf = np.load(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure5/simple_combined_{dataset}.npz')['']
        gpt2xlu_perf = np.load(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure5/gpt2xl_combined_{dataset}.npz')[fe]
        
        perf_diff = gpt2xlu_perf - simple_perf
        
        for i in range(1000):
            
            if dataset == 'pereira':
                
                gauss_perf_combined = np.full(subjects_arr_pereira.shape[0], fill_value=np.nan)
                
                for e in exp:
                
                    gauss_perf = np.load(f'/data/LLMs/brainscore/results_{dataset}/stats/{dataset}_gaussian-stats_layer_{i}_1_{e}.npz')[perf]
                    
                    gauss_perf_combined[non_nan_indices_dict[e]] = custom_add_2d(gauss_perf_combined[non_nan_indices_dict[e]],  
                                                                                    gauss_perf)
                    
                gauss_perf_combined = gauss_perf_combined[lang_indices]
                    
            else:
                
                gauss_perf_combined = np.load(f'/data/LLMs/brainscore/results_{dataset}/stats/{dataset}_gaussian-stats_layer_{i}_1.npz')[perf]
                
                
            all_gauss.append(gauss_perf_combined)
            
        
        all_gauss_np = np.stack(all_gauss)
        print(dataset, fe)
        _, _ = compute_voxel_pvalues(perf_diff, all_gauss_np)    

pereira 
Fraction of uncorrected p-values < 0.05: 11.599
Fraction of FDR-corrected p-values < 0.05: 0.487
pereira -mp
Fraction of uncorrected p-values < 0.05: 19.420
Fraction of FDR-corrected p-values < 0.05: 0.944
pereira -sp
Fraction of uncorrected p-values < 0.05: 17.989
Fraction of FDR-corrected p-values < 0.05: 0.782
fedorenko 
Fraction of uncorrected p-values < 0.05: 14.433
Fraction of FDR-corrected p-values < 0.05: 0.000
fedorenko -mp
Fraction of uncorrected p-values < 0.05: 20.619
Fraction of FDR-corrected p-values < 0.05: 4.124
fedorenko -sp
Fraction of uncorrected p-values < 0.05: 19.588
Fraction of FDR-corrected p-values < 0.05: 4.124
blank 
Fraction of uncorrected p-values < 0.05: 31.667
Fraction of FDR-corrected p-values < 0.05: 6.667
blank -mp
Fraction of uncorrected p-values < 0.05: 31.667
Fraction of FDR-corrected p-values < 0.05: 0.000
blank -sp
Fraction of uncorrected p-values < 0.05: 28.333
Fraction of FDR-corrected p-values < 0.05: 0.000
